In [1]:
import argparse
import json
from pathlib import Path
from brtm.config import CFG
from brtm.data.loaders import preprocess_docs, tok
from brtm.models.lda_init import init_lda_beta, shared_topics
from brtm.evaluation.metrics import evaluate_topn
from brtm.utils.features import XY, feat
from brtm.models.var_em_gpu import var_em_gpu_fixed
from brtm.models.lda_init import init_lda_beta, shared_topics
from brtm.models.lbfgs_gpu import lbfgs_gpu
from brtm.evaluation.mrr_ndcg import calculate_mrr_ndcg
from gensim import corpora
import warnings, random, pandas as pd, numpy as np, os
from tqdm import tqdm
import torch
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Set seeds for reproducibility
random.seed(CFG["seed"])
np.random.seed(CFG["seed"])
torch.manual_seed(CFG["seed"])

# Load English stopwords
STOP = set(stopwords.words("english"))

# Create output directory
OUT = Path("brtm_outputs")
OUT.mkdir(exist_ok=True, parents=True)

In [3]:
# 1. Data loading and preprocessing
print("Loading text data...")

# Load textual data
corpus, all_tok = {}, []
for tag, (fp, col, id_col) in CFG["files"].items():
    if tag.startswith(("C_", "train", "val", "test")):
        continue

    print(f"Reading {tag} data...")
    df = pd.read_csv(fp)

    # Handle special column names (e.g., "translation.comments")
    if "." in col:
        # Assume nested JSON-like column, parse manually
        docs = []
        for _, row in df.iterrows():
            try:
                # Try to parse JSON or use raw string
                text = str(row[col.split('.')[0]]) if '.' in col else str(row[col])
                docs.append(tok(text))
            except:
                docs.append([])
    else:
        docs = [tok(x) for x in tqdm(df[col].fillna(""), desc=f"tok {tag}")]

    corpus[tag] = docs
    all_tok.extend(docs)

print(f"Text data loaded. Total documents: {sum(len(docs) for docs in corpus.values())}")

# Build vocabulary
dictionary = corpora.Dictionary(all_tok)
dictionary.filter_extremes(5, 0.5)  # Remove rare and overly common tokens
Path(OUT / "vocab.json").write_text(json.dumps(dictionary.token2id))
V = len(dictionary)
print(f"Vocabulary size: {V}")


Loading text data...
Reading D_j data...


tok D_j: 100%|██████████| 10168/10168 [00:05<00:00, 1826.46it/s]


Reading D_li data...


tok D_li: 100%|██████████| 416/416 [00:01<00:00, 331.67it/s]


Reading A_i data...


...

Reading A_j data...


tok A_j: 100%|██████████| 9198/9198 [01:47<00:00, 85.58it/s] 


Reading B_i data...
Reading B_k data...


tok B_k: 100%|██████████| 858/858 [00:01<00:00, 452.31it/s]


Text data loaded. Total documents: 24527
Vocabulary size: 35176


In [4]:
# 2. Profile data preprocessing
from brtm.data.loaders import preprocess_docs, make_guest_features, make_host_features
from sklearn.preprocessing import MinMaxScaler

guest_raw = pd.read_csv(CFG["files"]["C_i"][0]).set_index("guest_id")
host_raw = pd.read_csv(CFG["files"]["C_k"][0]).set_index("host_id")

guest_prof = make_guest_features(guest_raw)
host_prof = make_host_features(host_raw)

# Normalize to [0, 1]
sc_g = MinMaxScaler().fit(guest_prof)
guest_p = pd.DataFrame(sc_g.transform(guest_prof), 
                       index=guest_prof.index, 
                       columns=guest_prof.columns)

sc_h = MinMaxScaler().fit(host_prof)
host_p = pd.DataFrame(sc_h.transform(host_prof), 
                      index=host_prof.index, 
                      columns=host_prof.columns)

print(f"Profile data processed: Guest={len(guest_p)}, Host={len(host_p)}")


Profile data processed: Guest=1879, Host=8930


In [5]:
# 3. Topic model initialization 
from brtm.models.lda_init import init_lda_beta, shared_topics

# Initialize LDA models for different document sources
beta_D_init, lda_D, bow_D = init_lda_beta(corpus["D_j"] + corpus["D_li"], dictionary, CFG)
beta_A_init, lda_A, bow_A = init_lda_beta(corpus["A_i"] + corpus["A_j"], dictionary, CFG)
beta_B_init, lda_B, bow_B = init_lda_beta(corpus["B_i"] + corpus["B_k"], dictionary, CFG)
# Match shared topics between different topic spaces
DA_D, DA_A = shared_topics(beta_D_init, beta_A_init, CFG["DA"])
AB_A, AB_B = shared_topics(beta_A_init, beta_B_init, CFG["AB"])

# Select disjoint (exclusive) topic sets
Dstar = [t for t in range(CFG["K"]) if t not in DA_D][:CFG["Dstar"]]
Astar = [t for t in range(CFG["K"]) if t not in DA_A + AB_A][:CFG["Astar"]]
Bstar = [t for t in range(CFG["K"]) if t not in AB_B][:CFG["Bstar"]]

# Construct initial topic-word distributions (phi) for D, A, B
phi0 = {
    "D": np.vstack([
        beta_D_init[Dstar],
        0.5 * beta_D_init[DA_D] + 0.5 * beta_A_init[DA_A]
    ]),
    "A": np.vstack([
        beta_A_init[Astar],
        0.5 * beta_D_init[DA_D] + 0.5 * beta_A_init[DA_A],
        0.5 * beta_A_init[AB_A] + 0.5 * beta_B_init[AB_B]
    ]),
    "B": np.vstack([
        beta_B_init[Bstar],
        0.5 * beta_A_init[AB_A] + 0.5 * beta_B_init[AB_B]
    ])
}

# Record topic index order for each block
order_D = Dstar + DA_D
order_A = Astar + DA_A + AB_A
order_B = Bstar + AB_B

print(f"Topic allocation - D:{len(order_D)}, A:{len(order_A)}, B:{len(order_B)}")


Building BOW: 100%|██████████| 10584/10584 [00:00<00:00, 26925.41it/s]


Training initial LDA...


Building BOW: 100%|██████████| 12638/12638 [00:03<00:00, 3539.97it/s]


Training initial LDA...


Building BOW: 100%|██████████| 1305/1305 [00:00<00:00, 15817.23it/s]


Training initial LDA...
Matching shared topics...
Matching shared topics...
Topic allocation - D:60, A:60, B:60


In [6]:
# 4. Initialize θ cache 

# Read ID lists from data files
l_id = pd.read_csv(CFG["files"]["D_j"][0], usecols=[CFG["files"]["D_j"][2]]).squeeze()
g_id = pd.read_csv(CFG["files"]["A_i"][0], usecols=[CFG["files"]["A_i"][2]]).squeeze()
h_id = pd.read_csv(CFG["files"]["B_k"][0], usecols=[CFG["files"]["B_k"][2]]).squeeze()

# Initialize θ cache (zero-initialized for each entity)
θD = dict(zip(l_id, np.zeros((len(l_id), len(order_D)))))
θA = dict(zip(g_id, np.zeros((len(g_id), len(order_A)))))
θB = dict(zip(h_id, np.zeros((len(h_id), len(order_B)))))

print(f"θ cache initialized: D={len(θD)}, A={len(θA)}, B={len(θB)}")


θ cache initialized: D=10168, A=3440, B=858


In [7]:
# 5. Load transaction data -------------------------------------------------------
print("Loading transaction data...")
train = pd.read_csv(CFG["files"]["train"][0])
val = pd.read_csv(CFG["files"]["val"][0])
test = pd.read_csv(CFG["files"]["test"][0])

print(f"Transaction data loaded: train={len(train)}, val={len(val)}, test={len(test)}")
print(f"Label distribution - train: {train['label'].value_counts().to_dict()}")
print(f"Label distribution - val: {val['label'].value_counts().to_dict()}")
print(f"Label distribution - test: {test['label'].value_counts().to_dict()}")


Loading transaction data...
Transaction data loaded: train=43979, val=6766, test=16915
Label distribution - train: {0: 41779, 1: 2200}
Label distribution - val: {0: 6429, 1: 337}
Label distribution - test: {0: 16069, 1: 846}


In [8]:
# 6. Preprocess document data 

# Preprocess all documents
docs_D_data, lengths_D = preprocess_docs(corpus["D_j"] + corpus["D_li"], dictionary)
docs_A_data, lengths_A = preprocess_docs(corpus["A_i"] + corpus["A_j"], dictionary)
docs_B_data, lengths_B = preprocess_docs(corpus["B_i"] + corpus["B_k"], dictionary)

print(f"Document preprocessing completed: D={len(docs_D_data)}, A={len(docs_A_data)}, B={len(docs_B_data)}")


Document preprocessing completed: D=10584, A=12638, B=1305


In [9]:
# 7. Main training loop -
beta = None

for outer in tqdm(range(CFG["em_outer"]), desc="Outer EM Loop"):
    print(f"\nOuter iteration {outer + 1}/{CFG['em_outer']}")

    # ---- Variational updates for three topic blocks ----------------------------------------
    print("Updating domain D topic model...")
    θD_mat, _, φD = var_em_gpu_fixed(docs_D_data, lengths_D, order_D, phi0["D"], lda_D, bow_D, "D", CFG)
    
    print("Updating domain A topic model...")
    θA_mat, _, φA = var_em_gpu_fixed(docs_A_data, lengths_A, order_A, phi0["A"], lda_A, bow_A, "A", CFG)
    
    print("Updating domain B topic model...")
    θB_mat, _, φB = var_em_gpu_fixed(docs_B_data, lengths_B, order_B, phi0["B"], lda_B, bow_B, "B", CFG)

    # Update θ cache
    print("Updating θ cache...")
    listing_ids = list(θD.keys())[:len(θD_mat)]
    guest_ids = list(θA.keys())[:len(θA_mat)]
    host_ids = list(θB.keys())[:len(θB_mat)]
    
    for i, lid in enumerate(listing_ids):
        if i < len(θD_mat):
            θD[lid] = θD_mat[i]
    
    for i, gid in enumerate(guest_ids):
        if i < len(θA_mat):
            θA[gid] = θA_mat[i]
    
    for i, hid in enumerate(host_ids):
        if i < len(θB_mat):
            θB[hid] = θB_mat[i]

    #Construct training features 
    print("Constructing training features...")
    X_train, y_train = XY(train,θD, θA, θB, guest_p, host_p, order_D, order_A, order_B)
    
    print(f"Training feature shape: {X_train.shape}, Label distribution: {np.bincount(y_train.astype(int))}")
    
    # LBFGS optimization
    print("Running LBFGS optimization...")
    beta = lbfgs_gpu(X_train, y_train, CFG, beta)

    # Validation evaluation
    print("Evaluating on validation set...")
    val_results = evaluate_topn(beta, val, CFG,θD, θA, θB, guest_p, host_p, order_D, order_A, order_B, topn_list=[1, 5, 10])
    
    print(f"Validation results: {val_results}")
    
    # Save intermediate checkpoint
    if (outer + 1) % 2 == 0:
        checkpoint_path = OUT / f"checkpoint_outer_{outer+1}.npz"
        np.savez(checkpoint_path,
                 theta_D=θD_mat, theta_A=θA_mat, theta_B=θB_mat,
                 phi_D=φD, phi_A=φA, phi_B=φB, beta=beta)
        print(f"Checkpoint saved: {checkpoint_path}")


Outer EM Loop:   0%|          | 0/5 [00:00<?, ?it/s]


Outer iteration 1/5
Updating domain D topic model...
Training domain D: K=60, V=35176, D=10584
Initial gamma stats - min:0.100, max:9.078, mean:0.267



D E-step: 100%|██████████| 40/40 [05:06<00:00,  7.70s/it]
                                                         

Outer 1: γ change=6377.455078, log-likelihood=-355205.96
φ range=[0.000000, 0.730409], γ range=[0.10, 1862.64]



D E-step: 100%|██████████| 40/40 [05:05<00:00,  7.55s/it]
                                                         

...

Updating θ cache...
Constructing training features...



build X: 100%|██████████| 43979/43979 [00:03<00:00, 12866.20it/s]


Training feature shape: (43979, 186), Label distribution: [41779  2200]
Running LBFGS optimization...
Evaluating on validation set...
Building test features...



Outer EM Loop: 100%|██████████| 5/5 [4:46:56<00:00, 3443.36s/it]

Hit Rate @ 1: 0.300
Hit Rate @ 5: 0.659
Hit Rate @ 10: 0.831
Validation results: {'HR@1': 0.2997032640949555, 'HR@5': 0.658753709198813, 'HR@10': 0.8308605341246291}


In [10]:
val_results = evaluate_topn(beta, val, CFG,θD, θA, θB, guest_p, host_p, order_D, order_A, order_B, topn_list=[1, 5, 10])

Building test features...


build X: 100%|██████████| 6766/6766 [00:00<00:00, 13037.26it/s]


Hit Rate @ 1: 0.300
Hit Rate @ 5: 0.659
Hit Rate @ 10: 0.831


In [12]:
# 8. Final evaluation (full Table 7 reproduction) 

# Full Top-N evaluation for Table 7
print("BRTM-Sample full evaluation results:")
topn_range = list(range(1, 11))  # 1 to 10, matching Table 7
final_results = evaluate_topn(beta, test, CFG, θD, θA, θB, guest_p, host_p, order_D, order_A, order_B, topn_list=topn_range)

# Standardized output format matching Table 7
print("\n" + "=" * 80)
print("Table 7 Reproduction Results - BRTM-Sample Hit Rate")
print("=" * 80)
print("Top-N | Hit Rate | Expected (from Table 7)")
print("-" * 50)

# Expected Hit Rate values in Table 7 for BRTM-Sample
expected_values = {
    1: 0.204, 2: 0.363, 3: 0.493, 4: 0.606, 5: 0.698,
    6: 0.778, 7: 0.839, 8: 0.885, 9: 0.920, 10: 0.945
}

for n in topn_range:
    hr = final_results[f'HR@{n}']
    expected = expected_values.get(n, 'N/A')
    diff = f"({hr - expected:+.3f})" if expected != 'N/A' else ""
    print(f"  {n:2d}  |  {hr:.3f}    |    {expected}  {diff}")


BRTM-Sample full evaluation results:
Building test features...


build X: 100%|██████████| 16915/16915 [00:01<00:00, 10684.59it/s]


Hit Rate @ 1: 0.300
Hit Rate @ 2: 0.434
Hit Rate @ 3: 0.529
Hit Rate @ 4: 0.566
Hit Rate @ 5: 0.645
Hit Rate @ 6: 0.696
Hit Rate @ 7: 0.769
Hit Rate @ 8: 0.800
Hit Rate @ 9: 0.843
Hit Rate @ 10: 0.857

Table 7 Reproduction Results - BRTM-Sample Hit Rate
Top-N | Hit Rate | Expected (from Table 7)
--------------------------------------------------
   1  |  0.300    |    0.204  (+0.096)
   2  |  0.434    |    0.363  (+0.071)
   3  |  0.529    |    0.493  (+0.036)
   4  |  0.566    |    0.606  (-0.040)
   5  |  0.645    |    0.698  (-0.053)
   6  |  0.696    |    0.778  (-0.082)
   7  |  0.769    |    0.839  (-0.070)
   8  |  0.800    |    0.885  (-0.085)
   9  |  0.843    |    0.92  (-0.077)
  10  |  0.857    |    0.945  (-0.088)


In [14]:
# 9. Compute additional evaluation metrics 
mrr, ndcg = calculate_mrr_ndcg(beta, test, CFG, θD, θA, θB, guest_p, host_p, order_D, order_A, order_B)
print(f"MRR: {mrr:.4f}")
print(f"NDCG@10: {ndcg:.4f}")


Computing MRR & NDCG: 100%|██████████| 846/846 [00:00<00:00, 3984.92it/s]

MRR: 0.4765
NDCG@10: 0.5628


In [15]:
# 10. Topic quality analysis 

# Map token IDs back to words
id2tok = {i: w for w, i in dictionary.token2id.items()}

def get_top_words(phi_matrix, n=10):
    """Get top-n words for each topic"""
    topics = []
    for k in range(phi_matrix.shape[0]):
        idx = phi_matrix[k].argsort()[-n:][::-1]
        words = [id2tok[i] for i in idx if i in id2tok]
        topics.append(words)
    return topics

# Extract top words per topic for each domain
topics_D = get_top_words(φD)
topics_A = get_top_words(φA)
topics_B = get_top_words(φB)

print(f"Topic extraction completed: D={len(topics_D)} topics, A={len(topics_A)} topics, B={len(topics_B)} topics")

# Display sample topics
print("\nTop 5 sample topics from domain D:")
for k in range(min(5, len(topics_D))):
    print(f"  Topic D{k}: {', '.join(topics_D[k][:8])}")

print("\nTop 5 sample topics from domain A:")
for k in range(min(5, len(topics_A))):
    print(f"  Topic A{k}: {', '.join(topics_A[k][:8])}")

print("\nTop 5 sample topics from domain B:")
for k in range(min(5, len(topics_B))):
    print(f"  Topic B{k}: {', '.join(topics_B[k][:8])}")


Topic extraction completed: D=60 topics, A=60 topics, B=60 topics

Top 5 sample topics from domain D:
  Topic D0: arena, metro, ziggo, dome, walking, bijlmer, park, afas
  Topic D1: room, cottage, park, please, b, accommodation, also, people
  Topic D2: west, vondelpark, de, br, baarsjes, restaurants, located, neighborhood
  Topic D3: years, appartment, feels, reviews, old, since, ago, like
  Topic D4: vondel, loads, sqm, hoofddorpplein, opportunity, prime, daylight, singles

Top 5 sample topics from domain A:
  Topic A0: hans, nick, us, great, wendie, stay, host, de
  Topic A1: mark, matthijs, wilma, us, olaf, breakfast, corina, sonja
  Topic A2: frank, von, sanne, mimi, de, us, house, place
  Topic A3: breakfast, morning, us, stay, great, reinout, delicious, place
  Topic A4: ruben, maria, jesse, stijn, great, us, yahav, place

Top 5 sample topics from domain B:
  Topic B0: thanks, see, pleasure, meet, big, ralf, review, de
  Topic B1: house, de, thanks, guests, like, also, review, d

In [16]:
# 11. Save full results and analysis 

# Helper function: convert NumPy types to native Python types for JSON
def convert_numpy_types(obj):
    """Recursively convert NumPy types to native Python types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj

# Construct final result dictionary
final_results_complete = {
    "hit_rates": convert_numpy_types(final_results),
    "mrr": float(mrr),
    "ndcg": float(ndcg),
    "theta_D": θD_mat, 
    "theta_A": θA_mat, 
    "theta_B": θB_mat,
    "phi_D": φD, 
    "phi_A": φA, 
    "phi_B": φB,
    "beta": beta,
    "config": convert_numpy_types(CFG)
}

# Save model parameters and metrics
results_path = OUT / "brtm_table7_complete_results.npz"
np.savez(results_path, **final_results_complete)

# Save topic analysis (fix JSON serialization for topic indices and word lists)
topics_data = {
    "topics_D": topics_D,
    "topics_A": topics_A, 
    "topics_B": topics_B,
    "shared_topics": {
        "DA_topics": int(len(DA_D)),
        "AB_topics": int(len(AB_A)),
        "D_specific": int(len(Dstar)),
        "A_specific": int(len(Astar)),
        "B_specific": int(len(Bstar))
    },
    "topic_allocation": {
        "order_D": [int(x) for x in order_D],
        "order_A": [int(x) for x in order_A],
        "order_B": [int(x) for x in order_B]
    }
}

topics_path = OUT / "topics_table7.json"
with open(topics_path, "w", encoding='utf-8') as f:
    json.dump(topics_data, f, indent=2, ensure_ascii=False)

# Save feature importance (β weights)
feature_importance = {
    "beta_weights": beta.tolist(),
    "feature_names": [
        f"D_topic_{i}" for i in range(len(order_D))
    ] + [
        f"A_topic_{i}" for i in range(len(order_A))
    ] + [
        f"B_topic_{i}" for i in range(len(order_B))
    ] + [
        "guest_years_since_join", "guest_n_verified_src", "guest_n_connected"
    ] + [
        "host_years_since_join", "host_n_verified_src", "host_is_superhost"
    ]
}

feature_path = OUT / "feature_importance.json"
with open(feature_path, "w") as f:
    json.dump(feature_importance, f, indent=2)

# Save evaluation summary report
report = {
    "model": "BRTM-Sample",
    "dataset": "Airbnb Transaction Data",
    "evaluation_date": pd.Timestamp.now().isoformat(),
    "performance": {
        "hit_rates": convert_numpy_types(final_results),
        "mrr": float(mrr),
        "ndcg_10": float(ndcg),
    },
    "model_config": convert_numpy_types(CFG),
    "data_statistics": {
        "train_size": int(len(train)),
        "val_size": int(len(val)),
        "test_size": int(len(test)),
        "vocabulary_size": int(V),
        "unique_listings": int(len(θD)),
        "unique_guests": int(len(θA)),
        "unique_hosts": int(len(θB))
    }
}

report_path = OUT / "evaluation_report.json"
with open(report_path, "w") as f:
    json.dump(report, f, indent=2)

# Summary of saved outputs
print(f"All results saved to: {OUT}")
print("Saved files:")
print(f"  - {results_path.name}: Complete results and model parameters")
print(f"  - {topics_path.name}: Topic word lists and structure")
print(f"  - {feature_path.name}: Feature importance (beta weights)")
print(f"  - {report_path.name}: Evaluation report summary")


All results saved to: brtm_outputs
Saved files:
  - brtm_table7_complete_results.npz: Complete results and model parameters
  - topics_table7.json: Topic word lists and structure
  - feature_importance.json: Feature importance (beta weights)
  - evaluation_report.json: Evaluation report summary


In [19]:
# 12. Summary of results and performance analysis 
print("\n" + "="*80)
print("BRTM-Sample Table 7 Reproduction Summary")
print("="*80)

print("\nKey Metrics:")
print(f"  - Hit@1: {final_results['HR@1']:.3f} (Target: 0.204)")
print(f"  - Hit@5: {final_results['HR@5']:.3f} (Target: 0.698)")
print(f"  - Hit@10: {final_results['HR@10']:.3f} (Target: 0.945)")
print(f"  - MRR: {mrr:.4f}")
print(f"  - NDCG@10: {ndcg:.4f}")

print("\nModel Configuration:")
print(f"  - Topic counts: D*={CFG['Dstar']}, A*={CFG['Astar']}, B*={CFG['Bstar']}")
print(f"  - Shared topics: DA={CFG['DA']}, AB={CFG['AB']}")
print(f"  - EM Iterations: Outer={CFG['em_outer']}, Inner={CFG['em_inner']}")
print(f"  - GPU Batch Size: {CFG['gpu_batch']}")
print(f"  - Device: {CFG['Device']}")

print("\nData Statistics:")
print(f"  - Training samples: {len(train)}")
print(f"  - Validation samples: {len(val)}")
print(f"  - Test samples: {len(test)}")
print(f"  - Vocabulary size: {V}")
print(f"  - Unique users: Guests={len(θA)}, Hosts={len(θB)}")
print(f"  - Unique listings: {len(θD)}")

print(f"\nResults saved to: {OUT.absolute()}")
print("You can use the saved files for further analysis and visualization.")



BRTM-Sample Table 7 Reproduction Summary

Key Metrics:
  - Hit@1: 0.300 (Target: 0.204)
  - Hit@5: 0.645 (Target: 0.698)
  - Hit@10: 0.857 (Target: 0.945)
  - MRR: 0.4765
  - NDCG@10: 0.5628

Model Configuration:
  - Topic counts: D*=38, A*=29, B*=51
  - Shared topics: DA=22, AB=9
  - EM Iterations: Outer=5, Inner=40
  - GPU Batch Size: 8192
  - Device: cuda

Data Statistics:
  - Training samples: 43979
  - Validation samples: 6766
  - Test samples: 16915
  - Vocabulary size: 35176
  - Unique users: Guests=3440, Hosts=858
  - Unique listings: 10168

Results saved to: /root/BRTM_PROJECT/brtm_outputs
You can use the saved files for further analysis and visualization.
